In [30]:
import pandas as pd
import re

In [31]:
directory = 'C:/Users/ac4296/OneDrive - Drexel University/Anubha & Dr. T - Shared/Urban Regulatory projects/Urban Final'
file_path = f"{directory}/Output/Step 1a Back(sources) transposed.csv"
file_path2 = f"{directory}/Output/Step 1b Back(sources) transposed - Regulation Indicator.csv"
output_path = f"{directory}/Output/Formating Urban"

In [32]:
data= pd.read_csv(file_path, header=0)
data['STATE'] = data['STATE'].str.strip()
# data.head(10)

In [33]:
# Add "Regulation Indicator"

data2= pd.read_csv(file_path2, header=0)
data2.drop(data.columns[0], axis=1, inplace=True)
data['STATE'] = data2['STATE']
data = pd.merge(data, data2, on=['STATE', 'Regulatory Type'], how='outer')
# data.head(50)

data2['STATE'].dtype

dtype('O')

In [34]:
# Extract "Regulatory Type" and "Regulatory Body" into separate columns

data.rename(columns={'Regulatory Type': 'Regulatory Type Full'}, inplace=True)

pattern = r'^(.*?)(?:\s*-\s*(AG|Other))?(?:\s+by\s+(AG|Other))?$'

matches = data['Regulatory Type Full'].str.extract(pattern, expand=False)
data['Regulatory Type'] = matches[0]
data['Regulatory Body'] = matches[1].combine_first(matches[2])


data['Regulatory Body'].fillna('', inplace=True)


In [35]:
# Add three more empty columns

data['Year legislation originally enacted'] = ''
data['Year legislation changed'] = ''
data['Type of Change'] = ''

# data.head(50)

In [36]:
import re

def extract_section_codes(notes):
    if isinstance(notes, str):
        # Step 1: Extract section codes following § and Section
        pattern_step1 = r'(?:§{1,2}|Section|amd )\s*([^\s;,\]]+)'
        matches_step1 = re.findall(pattern_step1, notes)

        if not matches_step1:
            # Step 2: Extract section codes following ILCS, K.S.A., M.C.L., V.A.M.S, N.J.S.A., N.J.A., N.D.C.C., RCWA
            pattern_step2 = r'(?:ILCS|K.S.A\.|M.C.L\.|V.A.M.S\.|MCA|N.J.S.A\.|N.J.A\.|N.D.C.C\.|NDCC|NDCC,|RCWA|LSA-R.S.|N.R.S\.|W.S.A.|NRS|Ala\. Code Sec\.\s)\s*([^\s;,\]]+)'
            matches_step2 = re.findall(pattern_step2, notes)
            return [match.rstrip(';,]:.') for match in matches_step2] if matches_step2 else ['']
        else:
            return [match.rstrip(';,]:.') for match in matches_step1]
    else:
        return ['']

# , |    


In [37]:
data['Section Code'] = data['Notes'].apply(lambda x: extract_section_codes(str(x)))

In [38]:
pd.set_option('display.max_colwidth', None)
# data[['Notes', 'Section Code']].head(60)

In [39]:
#Final

data = data.explode('Section Code')

data['Section Code'] = data['Section Code'].str.strip()

data = data.reset_index(drop=True)

# pd.set_option('display.max_colwidth', None)
# data[['Notes','Section Code']].head(50)

In [40]:
data['STATE'] = data['STATE'].str.strip()
data = data[~data['Section Code'].str.isalpha()]
# data.head()
# data.columns

In [41]:
desired_order = ['Unnamed: 0', 'STATE', 'Regulatory Type','Regulatory Type Full', 'Regulatory Body','Regulation Indicator','Notes',  
       'Year legislation originally enacted', 'Year legislation changed',
       'Type of Change', 'Section Code']
data = data[desired_order]
data.head()

,Unnamed: 0,STATE,Regulatory Type,Regulatory Type Full,Regulatory Body,Regulation Indicator,Notes,Year legislation originally enacted,Year legislation changed,Type of Change,Section Code
0,0,Alabama,STATE,STATE,,Alabama,Alabama,,,,
1,1,Alabama,Bifurcated,Bifurcated,,No,No Alabama Code § 13A-9-70 et seq.,,,,13A-9-70
2,2,Alabama,Registration Office,Registration Office,,NaN,Office of the Alabama Attorney General (Consumer Protection Division) http://www.ago.state.al.us/Page-Consumer-Protection-Consumer-Charities,,,,
3,3,Alabama,Sale of Assets,Sale of Assets - AG,AG,No,No,,,,
4,4,Alabama,Sale of Assets,Sale of Assets - Other,Other,No,No,,,,


In [42]:
data.to_excel(f'{output_path}/Section_codes_current.xlsx')

In [43]:
# Create "Regulation Indicator"

#  data['Regulation Indicator'] = ''

# def determine_regulation_indicator(note):
#     if pd.isna(note):
#         return 'No'
#     if isinstance(note, str):
#         note = note.lower()
#         if re.search(r'"[^"]*"', note):
#             return 'Yes'
#         if re.search(r'no,?[^"]*', note, re.IGNORECASE):
#             return 'No'
#         if note == 'yes':
#             return 'Yes'
#         elif note == 'no':
#             return 'No'
#     return 'Yes'

# data['Regulation Indicator'] = data['Notes'].apply(determine_regulation_indicator)



In [44]:
# import re

# def extract_section_codes(notes):
#     if isinstance(notes, str):
#         pattern = r'(?:§{1,2}|RCWA|Section|ILCS|K.S.A.|M.C.L.|V.A.M.S|N.J.S.A.|N.J.A..|N.D.C.C.|)\s*([^\s]+)'
#         matches = re.findall(pattern, notes)
#         return matches if matches else ['']
#     else:
#         return ['']

In [45]:
# import re

# def extract_section_codes(notes):
#     if isinstance(notes, str):
#         # Step 1
#         pattern_step1 = r'(?:§{1,2}|Section)\s*([^\s]+)'
#         matches_step1 = re.findall(pattern_step1, notes)

#         # Step 2
#         if not matches_step1:
            
#             pattern_step2 = r'(?:ILCS|K.S.A\.|M.C.L\.|V.A.M.S|N.J.S.A\.|N.J.A\.|N.D.C.C\.|RCWA|N.R.S\.|NRS)\s*([^\s]+)'
#             matches_step2 = re.findall(pattern_step2, notes)
#             return matches_step2 if matches_step2 else ['']
#         else:
#             return matches_step1
#     else:
#         return ['']